Siehe Init.ipynb für die notwendige Bibliotheken


Hier gibt es nichts zu sehen :-)


In [ ]:
import csv
import tabulate
from pvlib import iotools 
import time
import json
import os
import pandas as pd 
import calendar
import copy

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [ ]:
def loadSvensLastProfil(filename):
    lastprofil = []
    with open(filename) as csvdatei:
        csv_reader_object = csv.reader(csvdatei, delimiter=',')
        for row in csv_reader_object:
            row.pop(0)
            lastprofil.append([float(x) for x in row]) 
    assert len(lastprofil) % 24 == 0, f"Die Anzahl der Einträge muss durch 24 teilbar sein. Nur komplette Tage sind zulässig: {len(lastprofile)}" 
    days = int(len(lastprofil) / 24)

    tmp = chunkIt(lastprofil, days)
    lastprofil = []
    for l in tmp:
        out = []
        for v in l:
            out.extend(v)
        assert len(out) == 1440, f"Falsche Arraylänge: {len(out)}" 
        lastprofil.append(out)

    # Willkürliches Auffüllen des Lastprofiles mit bereits vorhanden Daten, bis ein Jahr voll ist
    pos = 0
    while (len(lastprofil) < 365):
        lastprofil.append(copy.deepcopy(lastprofil[pos % days]))
        pos += 1
    assert len(lastprofil) == 365, f"Aufbau des Lastprofiles: 365 Einträge mit jeweils 1440 Einträgen: {len(lastprofil)}"            

    jahresVerbrauch = sum([sum(v) for v in lastprofil])
    print("Stromverbrauch pro Jahr: " + str(round(jahresVerbrauch/1000)))
    return lastprofil

#
#
# Skaliert ein Lastprofil so, dass das Lastprofil pro Jahr den gewünschten kWh Wert erreicht
#
#
#
#
def lastprofilAnpassen(newKWH,lastprofil):
    lastprofilCheck(lastprofil)
    jahresVerbrauch = sum([sum(v) for v in lastprofil])
    faktor = (newKWH * 1000) / jahresVerbrauch
    for i in range(0,365):
        for j in range(0,1440):
            lastprofil[i][j] = lastprofil[i][j] * faktor
    return lastprofil

def getTaktFaktor(taktAn,taktAus):
    assert taktAn > 0 and taktAus >=0, "taktAn muss postiv sein und taktAus darf nicht negativ sein"
    taktwiederholungenProTag = 1440/(taktAn+taktAus)
    anZeit = taktAn * taktwiederholungenProTag
    ausZeit = taktAus * taktwiederholungenProTag
    faktor = 1440 / anZeit
    return faktor
    
#
#
# Fügt einem lastprofil den Verbrauch einer Wärmepumpe hinzu
# Hierzu wird der angebene Verbrauch der auf die einzelnen Monate verteilt.
# Innerhalb eines Monats wird der anteilige Verbrauch gleichmäßig auf jeden Messpunkt verteilt.
# 
#
def lastprofilWarmepumpe(energiebedarfKWH,taktAn=1440,taktAus=0):
    # Heizbedarf pro Monat https://de.statista.com/statistik/daten/studie/160067/umfrage/verbrauch-von-heizenergie-nach-monaten/
    verteilung = [0.161, 0.13, 0.125, 0.081, 0.035, 0.022, 0.017, 0.016, 0.052, 0.084, 0.122, 0.155]
    idx = 0
    lp = [[0]*1440 for x in range(0,365)]
    taktfaktor = getTaktFaktor(taktAn,taktAus)
    for monat in range(1,13):
        tageImMoment = calendar.monthrange(2001,monat)[1]
        energiebedarfProMin = energiebedarfKWH *1000 * verteilung[monat -1] / tageImMoment / 1440 * taktfaktor
        for j in range(0, tageImMoment):
            taktung = taktAn
            for m in range(0,1440):
                if (taktung > 0):
                    lp[idx][m] = energiebedarfProMin
                else:
                    lp[idx][m] = 0
                # Prüfen, ob neuer Takt begonnen werden muss
                taktung -= 1
                if (taktung <= -taktAus):
                    taktung = taktAn
                    
            idx += 1   
    return lp

            
def lastprofilCheck(lastprofil):
    assert len(lastprofil) == 365, f"365 Einträge mit jeweils 1440 Einträgen: {len(lastprofil)}"
    for i in range(0,365):
        assert len(lastprofil[i]) == 1440, f"365 Einträge mit jeweils 1440 Einträgen: {len(lastprofil[i])}"            
    


In [ ]:
def loadPVGISDataFromCVSFile(filename, year):
    pvgis = []
    with open(filename) as csvdatei:
        csv_reader_object = csv.reader(csvdatei, delimiter=',')
        for row in csv_reader_object:
            if len(row) == 0 or not row[0].startswith(str(year)):
                continue
            pvgis.append(row)
    assert len(pvgis) == 8760, f"8760 (24 Einträge pro Tag * 365 Tage)Einträge für ein Jahr erwartet: {len(pvgis)}" 
    
    # Umwandlung der Stundenwerten in rechnerische Minutenwerte
    out = []
    for tagesdaten in chunkIt(pvgis, 365):
        tag = []
        for stundeninfo in tagesdaten:
                tag.extend([float(stundeninfo[1])/60.0 for x in range(60)])
        out.append(tag)
    assert len(out) == 365, f"365 Einträge mit jeweils 1440 Einträgen: {len(out)}"            
    # Summe für Prüfung bilden
    summe = sum([ sum(x) for x in out ])
    print("PV Ertrag im Jahr:" + str(round(summe / 1000)))
    return out


def convertPVGISData(data, year):
    pvgis = []
    for index, row in data.iterrows():
        if (str(index).startswith(str(year))):
            pvgis.append(row['P'])
    assert len(pvgis) == 8760, f"8760 (24 Einträge pro Tag * 365 Tage)Einträge für ein Jahr erwartet: {len(pvgis)}" 

    # Umwandlung der Stundenwerten in rechnerische Minutenwerte
    out = []
    for tagesdaten in chunkIt(pvgis, 365):
        tag = []
        for stundeninfo in tagesdaten:
             tag.extend([float(stundeninfo)/60.0 for x in range(60)])
        out.append(tag)
    assert len(out) == 365, f"365 Einträge mit jeweils 1440 Einträgen: {len(out)}"   
    return out

def getFilename(d, year):
    filename="pvgis-"
    for x in sorted(d):
        filename += x + str(d[x])
    filename += str(year) + ".data"
    return filename    

def getPVGISData(cfg, year):
    os.makedirs('cache', exist_ok=True)
    # PV Daten von PVGIS abrufen
    pvdata = [  ]
    for pv in cfg:
        # Check for Cache file
        filename = os.path.join('cache',getFilename(pv, year))
        if (os.path.isfile(filename)):
            print("Cached: " + filename)
            data = pd.read_pickle(filename)
            data = convertPVGISData(data, year)
        else:
            print("Request from pvgis: " + filename)
            param = dict(pv)
            param["start"] = year
            param["end"] = year
            rawdata = iotools.get_pvgis_hourly(**param)
            rawdata[0].to_pickle(filename)
            data = convertPVGISData(rawdata[0], year)
                  
        pvdata.append(data)
        time.sleep(3.1)

    # Aufsummieren
    pvErtrag = [[0]*1440 for x in range(0,365)]
    for pv in pvdata:
        for tagIdx in range(0,365):
            for minIdx in range(0,1440):
                pvErtrag[tagIdx][minIdx] += pv[tagIdx][minIdx]


    # Summe für Prüfung bilden
    summe = sum([ sum(x) for x in pvErtrag ])
    print("PV Ertrag im Jahr:" + str(round(summe / 1000)))
    return pvErtrag
    

In [ ]:
   
def data2speech(v):
    out = ""
    out += "Netzbezug, " if v["netz"] > 0 else ""
    out += "Einspeisung, " if v["netz"] < 0 else ""
    out += "Ladung, " if v["battery"] > 0 else ""
    out += "Entladung, " if v["battery"] < 0 else ""
    return out

def stats(data,cfg):
    v = {"verbrauch": 0.0, "pv":0.0 , "netzbezug":0.0,"netzeinspeisung": 0.0, "batterieladung":0.0, "batterieendladung": 0.0}
    lasten = [ "v_" + l["name"] for l in cfg["last"]]
    for l in lasten:
        v[l] = 0
        
    for tage in data:
        for i in tage:
            v["verbrauch"] += i["verbrauch"]
            v["pv"] += i["pv"]
            if (i["netz"] > 0):
                v["netzbezug"] += i["netz"]
            else:
                v["netzeinspeisung"] -= i["netz"]

            if (i["battery"] > 0):
                v["batterieladung"] += i["battery"]
            else:
                v["batterieendladung"] -= i["battery"]
            
            for l in lasten:
                v[l] += i[l]
    for key in  v:
        v[key] = round(v[key]/1000)
    v["kapazität"] = cfg["battery"]["capacity"]
    v["vollzyklen"] = 0 if cfg["battery"]["capacity"] == 0 else round(v["batterieendladung"] / (cfg["battery"]["capacity"]/1000))
    return v
 
def statsMon(data,cfg):
    out = []
    idx = 0
    lasten = [ "v_" + l["name"] for l in cfg["last"]]
    for monat in range(1,13):
        tageProMonat = calendar.monthrange(2001,monat)[1]
        v = {"verbrauch": 0.0, "pv":0.0 , "netzbezug":0.0,"netzeinspeisung": 0.0, "batterieladung":0.0, "batterieendladung": 0.0}
        for l in lasten:
            v[l] = 0
        for j in range(0, tageProMonat):
                for i in data[idx]:
                    v["verbrauch"] += i["verbrauch"]
                    v["pv"] += i["pv"]
                    if (i["netz"] > 0):
                        v["netzbezug"] += i["netz"]
                    else:
                        v["netzeinspeisung"] -= i["netz"]

                    if (i["battery"] > 0):
                        v["batterieladung"] += i["battery"]
                    else:
                        v["batterieendladung"] -= i["battery"]
                    for l in lasten:
                        v[l] += i[l]
                idx = idx + 1
        for key in  v:
            v[key] = round(v[key]/1000)
        v["kapazität"] = cfg["battery"]["capacity"]
        v["vollzyklen"] = 0 if cfg["battery"]["capacity"] == 0 else round(v["batterieendladung"] / (cfg["battery"]["capacity"]/1000))
        v["monat"] = monat
        out.append(v)
    return out


Die eigentliche Routine zur Simulation:
* pvErtrag 2D-Array(365x1440) mit dem pvErtrag für jeden Tag des Jahres pro Minate 
* lastprofile 2D-Array(365x1440) mit dem Verbrauch für jeden Tag des Jahres pro Minute
* cfg Config-Objekt

In [ ]:
def simulate(cfg):
    assert len(cfg["last"]) > 0, f"Keine Last definiert"
    for last in cfg["last"]:
        lastprofile = last["profile"]
        name = last["name"]
        assert len(lastprofile) == 365, f"365 Einträge mit jeweils 1440 Einträgen! Last {name} {len(lastprofile)}"
        for i in range(0,365):
            assert len(last["profile"][i]) == 1440, f"365 Einträge mit jeweils 1440 Einträgen! Last: {name} Idx: {i} {len(lastprofil[i])}"            
    
    data = {
        "battery": {
            "level": 0
        }
    }
    jahresDaten = []
    for tagIdx in range(0,365):
        tagesData = []
        pv = cfg["pv"][tagIdx]
#        verbrauch = lastprofil[tagIdx]
        for minIdx in range(0,1440):
            v = { "stunde": minIdx // 60, "pv": pv[minIdx], "netz": 0, "battery": 0, "batterylevel": 0}
            gesamtVerbrauch = 0
            for last in cfg["last"]:
                v["v_"+ last["name"]] = last["profile"][tagIdx][minIdx]
                gesamtVerbrauch += last["profile"][tagIdx][minIdx]
            v["verbrauch"] = gesamtVerbrauch    
                
            # Differenz aus Verbrauch und PV-Ertrag
            balance = v["verbrauch"] - v["pv"]

            # Batterie Ladung/Enladugn bestimmen
            if (balance <= 0):
                # PV Produktion ist höher -- Strom verfügbar
                v["battery"] = min(cfg["battery"]["capacity"] - data["battery"]["level"], -balance, cfg["battery"]["chargelimit"])
            elif (balance > 0):
                # Stromverbrauch ist höher -- Strom wird benötigt
                v["battery"] = -min(data["battery"]["level"], balance, cfg["battery"]["discargelimit"])

            # Auf Basis der Battery Ladung/Enladung Werte anpassen
            data["battery"]["level"] += v["battery"]
            v["netz"] = balance + v["battery"]
            v["batterylevel"]= data["battery"]["level"]

            v["desc"] = data2speech(v)
            tagesData.append(v)
        jahresDaten.append(tagesData)
    return jahresDaten